# TP correction Atmosphérique (tutoriel traitement geospatial)


***
##### par Julien Meloche

Le but de ce TP est de pouvoir appliquer une correction atmosphérique sur des mesures de température de brillance ($T_B$). 
Ce TP vas vous permettre d'explorer des librairies très puissantes sur python.

1. installation de python et environnement anaconda
2. ouverture de fichier .tiff, netcdf avec xarray, pandas et numpy
3. traitement géospatiale (rasterio, geopandas)
4. correction atmosphérique


#### Création d'environnement avec conda (anaconda)
Ceci vas vous permettre d'installer des librairies pour différent projet sans créer des conflits. C'est une bonnne habitude d'avoir plusieurs environnement. Nous allons donc créer un environnment pour le traitement géospatiale.


utiliser votre barre de recherche windows pour ouvrir le anaconda prompt

dans le anaconda prompt ... <br><br>
crée un environnement virtuelle  avec la commande

```console
conda create --name geospatial
```
activer votre environnement
```console
conda activate geospatial
```
installer les librairies de base
```console
conda install numpy pandas xarray matplotlib
```
installer les librairies de traitement spatial
```console
conda install rasterio geopandas
```
